# Introduction au ML, IGPDE
## Alice Schoenauer Sebag, Jan. 2020
## Inspection générale des finances, MEF

Nous allons nous intéresser à un jeu de données biologiques.

## 1. Téléchargement

Il s'agit d'un jeu de données publiques [1]. Il a été pré-processé suivant la méthode décrite dans [2]. 
### Références: 
[1] [Caie et al., Molecular Cancer Therapeutics, 2010], [Ljosa et al., Nature Methods, 2012]

https://data.broadinstitute.org/bbbc/BBBC021/

[2] [Kang et al., Nature Biotech., 2016]

https://www.nature.com/articles/nbt.3419

In [ ]:
import os, pandas
import numpy as np
%matplotlib
if not os.path.isfile('IntroToML_TD.tar.gz'):
    print('Be patient while the data downloads')
    os.system('wget -c http://members.cbio.mines-paristech.fr/~aschoenauer/IntroToML_TD.tar.gz')
    os.system('tar -xvzf IntroToML_TD.tar.gz')
    print('Done!')

## 2. Exploration des données

Nous avons téléchargé deux fichiers :
* X (les features)
* Y (les classes de médicaments et d'autres métadonnées).

Jetons un coup d'oeil.

In [ ]:
# Lecture du fichier
Y = pandas.read_csv('IntroToML_TD/Caie_metadata.csv')

# Cela montre la "tête" du fichier : les 5 premières lignes
Y.head(5)

In [ ]:
# Combien de data points avons-nous ?
print('Nb de datapoints:',Y.shape[0])

# Combien de médicaments distincts ?
print("Nb de médicaments différents ", Y.Drug.nunique())

# **Question** Combien y a-t-il de "DrugClass" distinctes dans les données ?
print('Nb de classes ?')
# votre réponse

In [ ]:
# Regardons également la matrice de features
X = pandas.read_csv('IntroToML_TD/Caie_features.csv')
print(X.head())

# **Question** Combien de features dans ce jeu de données ?
print('Nb of features?')

Nous allons essayer de regarder les données à l'aide d'un graphe. Pour que cela soit plus informatif, nous allons commencer par transformer les données, en les projetant sur un nombre réduit d'axes informatifs. Il y a de nombreuses manières de faire cela : une analyse en composantes principales (ACP), un tSNE (t-distributed stochastic neighbor embedding), ou encore une umap (uniform manifold approximation), et beaucoup d'autres.

Pour aujourd'hui nous utiliserons une ACP, qui projette les données sur les combinaisons linéaires de features présentant la variance maximale des données. Plus d'infos sur l'ACP : https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales



In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as p
from mpl_toolkits.mplot3d import Axes3D

pca = PCA(n_components=3)
# Standardisation des données
nX = (X-np.mean(X,0))/np.std(X, 0)
# On trouve la projection et on projette
npX = pca.fit_transform(nX)

# Enfin, on représente les données dans le nouvel espace
f=p.figure(figsize=(18,18))
ax = f.gca(projection='3d')
ax.scatter(npX[:,0], npX[:,1], npX[:,2])
p.show()

Essayons d'ajouter de l'information sur les classes - plusieurs exemples en fonction des packages installés (le plus courant : *matplotlib*, est parfois moins intuitifs que des packages plus récents, par exemple *seaborn* et *altair*).

In [ ]:
# Les classes distinctes
distinct_drug_classes = sorted(list(Y.DrugClass.unique()))

# Choix de couleurs
color_names = ['red', 'blue', 'orange', 'magenta','black',  'green','0.5', '#bcbddc','yellow','cyan', '#a6cee3',
               '#dd1c77', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', 
               '#6a3d9a']
color_dict = dict(zip(distinct_drug_classes, color_names))

# J'assigne une couleur à chaque point en fonction de sa classe
Y['color'] = [color_dict[dc] for dc in Y.DrugClass]

# On replotte - amusez-vous à tourner le plot
f=p.figure(figsize=(18,18))
ax = f.gca(projection='3d')
ax.scatter(npX[:,0], npX[:,1], npX[:,2], color=Y.color)
p.show()

## Remarque : les plots en 3D sont souvent trompeurs, car le résultat dépend de l'angle selon lequel on se place.

Ajoutons aussi une légende

In [ ]:
f, subplots=p.subplots(1, 3, figsize=(15,22))
subplots[0].scatter(npX[:,0], npX[:,1], color=Y.color, alpha=0.5)
subplots[0].set_xlabel('Principal component 1')
subplots[0].set_ylabel('Principal component 2')
subplots[1].scatter(npX[:,0], npX[:,2], color=Y.color, alpha=0.5)
subplots[1].set_xlabel('Principal component 1')
subplots[1].set_ylabel('Principal component 3')
subplots[2].scatter(npX[:,1], npX[:,2], color=Y.color, alpha=0.5)
subplots[2].set_xlabel('Principal component 2')
subplots[2].set_ylabel('Principal component 3')
# Ajout de la légende
for i, drug_class in enumerate(distinct_drug_classes):
    subplots[2].scatter(0,0, color=color_names[i], label=drug_class)
subplots[2].legend()

p.show()

Si le package *seaborn* est installé :

In [ ]:
try:
    import seaborn as sn
except ModuleNotFoundError:
    print("Seaborn n'est pas installé, passons.")
else:
    for k in range(3):
        Y['pc_{}'.format(k)] = npX[:,k]
    sn.pairplot(Y[[*['pc_{}'.format(k) for k in range(3)], 'DrugClass']], hue='DrugClass', palette=color_dict, 
                corner=True)

**Question** En inspectant Y, on remarque qu'on a aussi l'information concernant la nature expérimentale du datapoint : contrôle positif, négatif, ou expérience. Pouvez-vous refaire les plots ci-dessus en ajoutant cette information ?

## 3. Classification

Pour cette étape, on se restreint aux datapoints dont la classe est connue, mais également à la dose pour laquelle la substance utilisée est active.

In [ ]:
# Je repère les points inutiles
wh = np.where((Y.DrugClass=='Unknown')|(Y.Active==False))[0]

# Je supprime les lignes correspondantes dans les données
active_Y = Y.drop(wh, axis=0)
active_X = np.delete(X.values, wh, 0)

print('Nb de points restants : ', active_X.shape[0])

**Question** Peut-on afficher le nombre de points par classe ? Quelle est la difficulté par rapport aux difficultés énoncées tout à l'heure ?

### a. Nous allons tester les forêts aléatoires, en utilisant le package scikit-learn. Toute la documentation se trouve sur le site : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
# On choisit les forêts aléatoires comme hypothesis space
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

# On apprend le modèle sur les données :
model.fit(active_X, active_Y.DrugClass)

Maintenant on peut regarder l'erreur du modèle, et notamment la matrice de confusion. L'entrée (i, j) de cette matrice indiquera combien de points avec le vrai label *i* ont été prédits dans la class *j*.

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score

predicted_Y = model.predict(active_X)
score = recall_score(active_Y.DrugClass, predicted_Y, average='micro')
print("Model accuracy: ", score)

conf_mat = confusion_matrix(active_Y.DrugClass, predicted_Y)
print(conf_mat)

Plutôt pas mal ! Mais est-on sûr de ne pas être en train d'overfitter sérieusement ?

**Question** Séparez les données en un ensemble d'entraînement et un ensemble de test, pour entraîner un nouveau modèle sur l'entraînement et obtenir une meilleure estimation de l'erreur qu'il aurait sur des données qu'il n'a jamais vues. On pourra utiliser la classe suivante : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

### b. Avec un petit réseau de neurones. On pourra utiliser la classe suivante : https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

Pour aller plus loin, on peut conseiller PyTorch (https://pytorch.org/).

In [ ]:
from sklearn.neural_network import MLPClassifier

# Ici il est important de normaliser les données avant de les utiliser.
nactive_X = (active_X - np.mean(active_X, 0))/np.std(active_X, 0)

model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', batch_size=200, verbose=10)
model.fit(nactive_X, active_Y.DrugClass)

**Question** Quelle est l'accuracy du modèle ?

**Question** Essayez de faire varier le nombre de couches et le nombre de neurones de chaque couche.